In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import gc
from sklearn.metrics import confusion_matrix, roc_curve 
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_colwidth', None)
pd.set_option('future.no_silent_downcasting', True)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# df = pd.read_csv("output_adam.csv")
# df_label = pd.read_csv("mirai_labels.csv")
# df["alert"] = df_label
# df.head()

/kaggle/input/aaaaaaaaaa/Video_Injection_X_train.csv
/kaggle/input/aaaaaaaaaa/ARP_MitM_y_val.csv
/kaggle/input/aaaaaaaaaa/OS_Scan_X_train.csv
/kaggle/input/aaaaaaaaaa/ARP_MitM_X_train.csv
/kaggle/input/aaaaaaaaaa/SSDP_Flood_X_train.csv
/kaggle/input/aaaaaaaaaa/mirai_X_train.csv
/kaggle/input/aaaaaaaaaa/mirai_y_val.csv
/kaggle/input/aaaaaaaaaa/SYN_DoS_X_train.csv
/kaggle/input/aaaaaaaaaa/Fuzzing_X_val.csv
/kaggle/input/aaaaaaaaaa/OS_Scan_y_val.csv
/kaggle/input/aaaaaaaaaa/Fuzzing_y_train.csv
/kaggle/input/aaaaaaaaaa/ARP_MitM_X_val.csv
/kaggle/input/aaaaaaaaaa/Video_Injection_y_val.csv
/kaggle/input/aaaaaaaaaa/Active_Wiretap_y_val.csv
/kaggle/input/aaaaaaaaaa/SSL_Renegotiation_X_val.csv
/kaggle/input/aaaaaaaaaa/ARP_MitM_y_train.csv
/kaggle/input/aaaaaaaaaa/SYN_DoS_X_val.csv
/kaggle/input/aaaaaaaaaa/OS_Scan_X_val.csv
/kaggle/input/aaaaaaaaaa/Active_Wiretap_y_train.csv
/kaggle/input/aaaaaaaaaa/SSDP_Flood_y_val.csv
/kaggle/input/aaaaaaaaaa/mirai_y_train.csv
/kaggle/input/aaaaaaaaaa/SYN_DoS_

In [2]:
import glob
X_train_file_names = (glob.glob("/kaggle/input/aaaaaaaaaa/*X_train.csv"))
y_train_file_names = (glob.glob("/kaggle/input/aaaaaaaaaa/*y_train.csv"))

In [3]:
X_train_file_names

['/kaggle/input/aaaaaaaaaa/Video_Injection_X_train.csv',
 '/kaggle/input/aaaaaaaaaa/OS_Scan_X_train.csv',
 '/kaggle/input/aaaaaaaaaa/ARP_MitM_X_train.csv',
 '/kaggle/input/aaaaaaaaaa/SSDP_Flood_X_train.csv',
 '/kaggle/input/aaaaaaaaaa/mirai_X_train.csv',
 '/kaggle/input/aaaaaaaaaa/SYN_DoS_X_train.csv',
 '/kaggle/input/aaaaaaaaaa/Fuzzing_X_train.csv',
 '/kaggle/input/aaaaaaaaaa/SSL_Renegotiation_X_train.csv',
 '/kaggle/input/aaaaaaaaaa/Active_Wiretap_X_train.csv']

In [4]:
df = pd.DataFrame()
num_rows = 0
for X_train , y_train in zip(X_train_file_names, y_train_file_names):
    print(X_train , y_train)
    df_aux = pd.read_csv(X_train,low_memory=False)#.reset_index()
    num_rows += df_aux.shape[0]
    df_labels = pd.read_csv(y_train,low_memory=False)#.reset_index()
    # df_aux = pd.concat((df_aux,df_labels), axis=1)
    df_aux["alert"] = df_labels
    df = pd.concat((df,df_aux), axis=0)
    print(num_rows,df_aux.shape[0])
df.reset_index(inplace = True)
del df_aux, df_labels
gc.collect()
print(num_rows, df.shape[0])


/kaggle/input/aaaaaaaaaa/Video_Injection_X_train.csv /kaggle/input/aaaaaaaaaa/Fuzzing_y_train.csv
1977920 1977920
/kaggle/input/aaaaaaaaaa/OS_Scan_X_train.csv /kaggle/input/aaaaaaaaaa/ARP_MitM_y_train.csv
3336200 1358280
/kaggle/input/aaaaaaaaaa/ARP_MitM_X_train.csv /kaggle/input/aaaaaaaaaa/Active_Wiretap_y_train.csv
5339613 2003413
/kaggle/input/aaaaaaaaaa/SSDP_Flood_X_train.csv /kaggle/input/aaaaaaaaaa/mirai_y_train.csv
8601425 3261812
/kaggle/input/aaaaaaaaaa/mirai_X_train.csv /kaggle/input/aaaaaaaaaa/SYN_DoS_y_train.csv
9212733 611308
/kaggle/input/aaaaaaaaaa/SYN_DoS_X_train.csv /kaggle/input/aaaaaaaaaa/SSL_Renegotiation_y_train.csv
11429753 2217020
/kaggle/input/aaaaaaaaaa/Fuzzing_X_train.csv /kaggle/input/aaaaaaaaaa/Video_Injection_y_train.csv
13225064 1795311
/kaggle/input/aaaaaaaaaa/SSL_Renegotiation_X_train.csv /kaggle/input/aaaaaaaaaa/SSDP_Flood_y_train.csv
14991120 1766056
/kaggle/input/aaaaaaaaaa/Active_Wiretap_X_train.csv /kaggle/input/aaaaaaaaaa/OS_Scan_y_train.csv
168140

In [5]:
num_rows, df.shape

(16814071, (16814071, 75))

In [5]:
df.columns

Index(['index', 'frame.number', 'frame.len', 'frame.time', 'frame.time_epoch',
       'frame.protocols', 'eth.src', 'eth.dst', 'eth.type', 'ip.src', 'ip.dst',
       'ip.len', 'ip.ttl', 'ip.flags', 'ip.frag_offset', 'ip.proto',
       'ip.version', 'ip.dsfield', 'ip.checksum', 'tcp.srcport', 'tcp.dstport',
       'tcp.len', 'tcp.seq', 'tcp.ack', 'tcp.flags', 'tcp.flags.syn',
       'tcp.flags.ack', 'tcp.flags.fin', 'tcp.flags.reset', 'tcp.window_size',
       'tcp.checksum', 'tcp.stream', 'udp.srcport', 'udp.dstport',
       'udp.length', 'udp.checksum', 'icmp.type', 'icmp.code', 'icmp.checksum',
       'http.request.method', 'http.request.uri', 'http.request.version',
       'http.request.full_uri', 'http.response.code', 'http.user_agent',
       'http.content_length_header', 'http.content_type', 'http.cookie',
       'http.host', 'http.referer', 'http.location', 'http.authorization',
       'http.connection', 'dns.qry.name', 'dns.qry.type', 'dns.qry.class',
       'dns.flags.response

In [6]:
df["frame.time"].iloc[1]

'Feb 20, 2017 14:22:01.035897000 Środkowoeuropejski czas stand.'

In [4]:
def ordinalize_packets(packet_data, column="ip"):
    src_col = f"{column}.src"
    dst_col = f"{column}.dst"
    if 'frame.number' not in packet_data.columns or src_col not in packet_data.columns or dst_col not in packet_data.columns:
        raise ValueError("DataFrame must contain 'frame.number' and  columns")

    out_src = packet_data.reset_index().groupby([src_col])['index'].min()
    out_dst = packet_data.reset_index().groupby([dst_col])['index'].min() + 0.5
    out = pd.concat([out_src, out_dst], axis=0).sort_values()
    out = out[~out.index.duplicated(keep='first')]
    out.iloc[:] = out.reset_index(name="ip").index
    out = out.astype(int)
    mappings = out.to_dict()
    return mappings
    #packet_data.loc[:, src_col] = packet_data[src_col].replace(mappings).astype(int)
    #packet_data.loc[:, dst_col] = packet_data[dst_col].replace(mappings).astype(int)

In [5]:
features = ["frame.number", "frame.len", 'frame.time', 'eth.src', 'eth.dst', 'ip.src', 'ip.dst', 'ip.len', 'ip.ttl',
            'tcp.srcport', 'tcp.dstport',
            'tcp.flags.ack', 'tcp.flags.fin', 'tcp.flags.reset', "alert", "udp.srcport", "udp.dstport"]
df = df[features]
df["frame.len"] /= 65535
df["ip.len"] /= 65535
df["ip.ttl"] /= 255
df["tcp.srcport"] /= 65535
df["tcp.dstport"] /= 65535
df["udp.srcport"] /= 65535
df["udp.dstport"] /= 65535
df["frame.time"] = df["frame.time"].str.replace(" Środkowoeuropejski czas letni", "")#mój PCAP był po POlsku Pudzian byłby dumny :peepoGlad# CEST
df["frame.time"] = df["frame.time"].str.replace(" Środkowoeuropejski czas stand.", "")
df["frame.time"] = df["frame.time"].str.replace(" CEST", "")
df["frame.time"] = pd.to_datetime(df["frame.time"])
df['dow_sin'] = np.sin(2 * np.pi * df["frame.time"].dt.dayofweek / 6)
df['dow_cos'] = np.cos(2 * np.pi * df["frame.time"].dt.dayofweek / 6)
df['hour_sin'] = np.sin(2 * np.pi * df["frame.time"].dt.hour / 24.0)
df['hour_cos'] = np.cos(2 * np.pi * df["frame.time"].dt.hour / 24.0)
df["minute_sin"] = np.sin(2 * np.pi * df["frame.time"].dt.minute / 60)
df["minute_cos"] = np.cos(2 * np.pi * df["frame.time"].dt.minute / 60)
df["second_sin"] = np.sin(2 * np.pi * df["frame.time"].dt.second / 60)
df["second_cos"] = np.cos(2 * np.pi * df["frame.time"].dt.second / 60)
df["ms_sin"] = np.sin(2 * np.pi * df["frame.time"].dt.microsecond / 1000000)
df["ms_cos"] = np.cos(2 * np.pi * df["frame.time"].dt.microsecond / 1000000)
df = df.fillna(0)  # only tcp is left
df = df.drop(columns=["frame.time"])

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sin
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in cos
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sin
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in cos
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sin
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in cos
  result = getattr(uf

In [9]:
df.head()

,frame.number,frame.len,eth.src,eth.dst,ip.src,ip.dst,ip.len,ip.ttl,tcp.srcport,tcp.dstport,...,dow_sin,dow_cos,hour_sin,hour_cos,minute_sin,minute_cos,second_sin,second_cos,ms_sin,ms_cos
0,412812.0,0.023102,10:4f:a8:7e:7d:b5,00:a0:de:f1:88:6e,192.168.2.12,192.168.100.5,0.022889,0.25098,0.0,0.0,...,0.0,1.0,-0.5,-0.866025,0.913545,-0.406737,0.978148,-0.207912,-0.962926,-0.269765
1,604190.0,0.023102,10:4f:a8:70:22:7e,00:a0:de:f1:88:6e,192.168.2.13,192.168.100.5,0.022889,0.25098,0.0,0.0,...,0.0,1.0,-0.5,-0.866025,0.743145,-0.669131,0.104528,0.994522,0.223640,0.974672
2,13972.0,0.023102,10:4f:a8:7e:7d:b5,00:a0:de:f1:88:6e,192.168.2.12,192.168.100.5,0.022889,0.25098,0.0,0.0,...,0.0,1.0,-0.5,-0.866025,0.978148,0.207912,-0.669131,-0.743145,-0.109666,-0.993969
3,1466178.0,0.023102,10:4f:a8:70:22:7e,00:a0:de:f1:88:6e,192.168.2.13,192.168.100.5,0.022889,0.25098,0.0,0.0,...,0.0,1.0,-0.5,-0.866025,-0.406737,-0.913545,0.994522,-0.104528,-0.786785,0.617227
4,1733382.0,0.023102,10:4f:a8:7e:7d:b5,00:a0:de:f1:88:6e,192.168.2.12,192.168.100.5,0.022889,0.25098,0.0,0.0,...,0.0,1.0,-0.5,-0.866025,-0.743145,-0.669131,0.406737,0.913545,-0.999940,-0.010945


In [6]:
from torch.utils.data import Dataset
from tqdm import trange
from multiprocessing import Pool


class NetworkDataset(Dataset):
    def __init__(self, data, window=128):
        p = Pool(processes=1)
        self.data = data
        self.window = window
        self.mappings_ip = p.map(self.ordinal_ip, trange(len(self)))
        self.mappings_eth = p.map(self.ordinal_eth, trange(len(self)))
        p.close()
        p.join()
            
    def ordinal_ip(self, i):
        return ordinalize_packets(self.data[i:i+self.window], "ip")
    def ordinal_eth(self, i):
        return ordinalize_packets(self.data[i:i+self.window], "eth")
    
    def __getitem__(self, index):
        packets = self.data[index:index + self.window]
        labels = packets['alert'].to_numpy().astype(np.float32)
        packets.loc[:, "eth.src"] = packets["eth.src"].replace(self.mappings_eth[index]).astype(int)
        packets.loc[:, "eth.dst"] = packets["eth.dst"].replace(self.mappings_eth[index]).astype(int)
        packets.loc[:, "ip.src"] = packets["ip.src"].replace(self.mappings_ip[index]).astype(int)
        packets.loc[:, "ip.dst"] = packets["ip.dst"].replace(self.mappings_ip[index]).astype(int)
        packets = packets.drop(columns=['alert', "frame.number"]).fillna(0)
        return (packets[["eth.src", "eth.dst"]].astype(int).values, packets[["ip.src", "ip.dst"]].astype(int).values,
                packets.drop(columns=["ip.src", "ip.dst", "eth.src", "eth.dst"]).values.astype("float32"), labels)

    def __len__(self):
        return self.data.shape[0] - self.window

In [19]:
nds = NetworkDataset(df, 1024)
nds[12][1]

 36%|███▌      | 6086769/16813047 [01:02<00:31, 339459.54it/s] Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
  File "/usr/lib/python3.11/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
           ^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_35/3743630211.py", line 9, in ordinalize_packets
    out = pd.concat([out_src, out_dst], axis=0).sort_values()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.11/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
                    ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multipro

KeyboardInterrupt: 

  File "properties.pyx", line 36, in pandas._libs.properties.CachedProperty.__get__
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/internals/blocks.py", line 228, in _consolidate_key
    return self._can_consolidate, self.dtype.name
                                  ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/numpy/core/_dtype.py", line 346, in _name_get
    def _name_get(dtype):
    
KeyboardInterrupt


In [7]:
from torch.nn.utils.parametrizations import weight_norm


class ResBlock(nn.Module):
    def __init__(self, channels, kernel_size, dilation):
        super(ResBlock, self).__init__()
        self.conv1 = weight_norm(
            nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=kernel_size, dilation=dilation))
        self.conv2 = weight_norm(
            nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=kernel_size, dilation=dilation))
        self.pad = nn.ZeroPad1d((kernel_size - 1) // 2 * dilation)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        t = self.pad(x)
        t = F.relu(self.conv1(t))
        t = self.pad(t)
        t = F.relu(self.conv2(t))
        return self.dropout(t) + x

In [8]:
class TCN(nn.Module):
    def __init__(self, channels, kernel_size, blocks):
        super(TCN, self).__init__()
        self.blocks = nn.Sequential(*[ResBlock(channels, kernel_size, 2**i) for i in range(blocks)])
        
    def forward(self, x):
        return self.blocks(x)

In [16]:
rb =TCN(5, 5, 2)
rb(torch.randn(2, 5, 50))

tensor([[[ 0.4930,  2.8734,  0.7040,  1.4007,  0.5570,  0.8242,  0.5057,
           0.6803,  0.7938,  0.3710, -0.5096,  0.6751, -0.0863, -0.2357,
           0.1171,  1.3098, -0.0148,  1.3838, -0.1150, -1.2456, -0.6741,
           0.1877, -0.4860,  1.2179,  2.0754, -0.3600, -0.2070, -0.7151,
           0.6399, -0.0777,  0.3021, -0.3402,  0.4434,  1.5218,  0.5087,
           0.1899, -0.6611, -0.4585,  2.4599,  0.4934,  0.4422,  0.5624,
           2.2803, -1.5272,  0.7496, -0.0379,  1.2823,  0.8802, -0.7981,
           0.6152],
         [ 1.8333, -0.5736, -1.3496,  1.0371,  1.0337,  0.0684, -0.2696,
           2.1811,  0.2497,  1.6957, -2.0771,  0.3720,  2.0790, -0.3636,
           2.0944, -0.1222,  2.0948,  0.6292,  1.3397,  0.6612, -0.6909,
           1.0906, -1.5022,  0.5549, -0.8636, -1.5973, -0.5547,  1.2952,
           1.8265,  0.9957,  2.0427, -0.8961,  1.0709, -0.0629,  0.7529,
          -2.8566,  0.5028, -0.9615, -0.4447, -1.2141,  0.3986,  0.6234,
          -1.0600, -0.6818,  0.

In [9]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.eth_embed = nn.Embedding(128, 32)
        self.ip_embed = nn.Embedding(128, 32)
        self.total_embed = nn.Linear(148, 64)
        self.conv = TCN(128, 5, 3)
        self.projection = nn.Linear(64, 1)

    def forward(self, eth, ip, packet):
        B = packet.size(0)
        L = packet.size(1)
        eth = self.eth_embed(eth).view(B, L, -1)
        ip = self.ip_embed(ip).view(B, L, -1)
        pk_embed = self.total_embed(torch.cat((eth, ip, packet), dim=-1))
        processed = nn.functional.relu(self.conv(pk_embed))
        preds = self.projection(processed).squeeze(-1)
        return nn.functional.sigmoid(preds)

In [16]:
gc.collect()

1004

In [ ]:
dataset = NetworkDataset(df, 128) # preparation of dataset



  0%|          | 0/16813943 [00:00<?, ?it/s]

  0%|          | 1/16813943 [00:00<611:48:06,  7.63it/s]

  1%|▏         | 217162/16813943 [00:00<00:14, 1132828.69it/s]

  3%|▎         | 490888/16813943 [00:00<00:08, 1817898.92it/s]

  5%|▍         | 802559/16813943 [00:00<00:06, 2309952.90it/s]

  7%|▋         | 1126939/16813943 [00:00<00:05, 2637856.12it/s]

  9%|▊         | 1465563/16813943 [00:00<00:05, 2887839.21it/s]

 11%|█         | 1807198/16813943 [00:00<00:04, 3058545.40it/s]

 13%|█▎        | 2161534/16813943 [00:00<00:04, 3211594.41it/s]

 15%|█▌        | 2522379/16813943 [00:00<00:04, 3334924.00it/s]

 17%|█▋        | 2880250/16813943 [00:01<00:04, 3409840.23it/s]

 19%|█▉        | 3250006/16813943 [00:01<00:03, 3497614.60it/s]

 21%|██▏       | 3603641/16813943 [00:01<00:03, 3509328.79it/s]

 24%|██▎       | 3973671/16813943 [00:01<00:03, 3567063.71it/s]

In [15]:
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm
from accelerate import Accelerator

accelerator = Accelerator(mixed_precision="bf16")
model = Model()
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True, num_workers=1, pin_memory=True,
                        persistent_workers=True)
optimizer = Adam(model.parameters(), lr=1e-4)
w = torch.tensor([5.28], dtype=torch.float32, device="cuda")
loss_fn = nn.BCELoss(weight=w)

model, dataloader, optimizer = accelerator.prepare(model, dataloader, optimizer)

for epoch in range(20):
    count = 0
    total = 0

    for eth, ip, pkt, labels in tqdm(dataloader):
        optimizer.zero_grad()
        pkt = model(eth, ip, pkt)
        loss = loss_fn(pkt, labels)
        count += 1
        total += loss.item()
        loss.backward()
        optimizer.step()

    print("Epoch {} Loss {}".format(epoch, total / count))

AttributeError: partially initialized module 'torch._dynamo' has no attribute 'external_utils' (most likely due to a circular import)

In [ ]:
X_val_file_names = (glob.glob("/kaggle/input/aaaaaaaaaa/*X_val.csv"))
y_val_file_names = (glob.glob("/kaggle/input/aaaaaaaaaa/*y_val.csv"))

In [ ]:
df_val = pd.DataFrame()

num_rows = 0
for X_val_name , y_val_name in zip(X_val_file_names, y_val_file_names):
    print(X_val_name , y_val_name)
    df_val_aux = pd.read_csv(X_val_name, low_memory=False)#.reset_index()
    df_val_labels_aux = pd.read_csv(y_val_name, low_memory=False)#.reset_index()
    df_val_aux["alert"] = df_val_labels_aux
    df_val = pd.concat((df_val,df_val_aux), axis=0)
    print(num_rows,df_val_aux.shape[0])
df_val.reset_index(inplace = True)
del df_val_aux, df_val_labels_aux
gc.collect()
print(df_val_labels.shape[0], df_val.shape[0])


In [ ]:
dataset_val = NetworkDataset(df_val, 128)

In [39]:
ip, eth, pkt, labels = dataset_val
ip = torch.from_numpy(ip).cuda().unsqueeze(0)
eth = torch.from_numpy(eth).cuda().unsqueeze(0)
pkt = torch.from_numpy(pkt).cuda().unsqueeze(0)
labels = torch.from_numpy(labels).cuda().unsqueeze(0)

pred = model(ip, eth, pkt)
print(pred)
print(pred > 0.5)
labels

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
          True,  True, False, False, False,  True, False, False, False, False,
          True,  True,  True,  True, False, False, F

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
         1., 1.]], device='cuda:0')

In [ ]:
pred = pred.reshape(-1, 1)
labels =  labels.reshape(-1, 1)

In [68]:
loss_fn(pred, labels)

tensor(0.7049, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)

In [ ]:
#confusion matrix
cm = confusion_matrix(labels, pred)
num = labels.shape[0]
norm = cm/num
# Plot with Seaborn
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='.4f', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Incident Grade Prediction')
plt.show()

In [ ]:
#ROC Curve
fpr, tpr, thresholds = roc_curve(labels, pred)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.4f)' % roc_auc_score(y_test, y_pred))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()